In [ ]:
import numpy as np
import json
import pandas as pd
import csv

In [18]:
scene = "/home/shivam/machine_learning/LLFF/data/only_car/"

In [19]:
with open(scene+"/base_cam.json",'r') as f:
    base_cam = json.load(f)

In [20]:
names=["POINT3D_ID", "X", "Y", "Z", "R", "G", "B", "ERROR"]
df = []
with open(scene + "/colmap_text/points3D.txt", "r") as fp:
    reader = csv.reader(filter(lambda row: row[0]!='#', fp), delimiter=' ')
    for row in reader:
        df.append(row[:len(names)])
df = pd.DataFrame(df, columns=names)
df = df.set_index(names[:1])
origin = np.float64(df[['X','Y','Z']].values).mean(axis=0)

In [21]:
names=["IMAGE_ID", "QW", "QX", "QY", "QZ", "TX", "TY", "TZ", "CAMERA_ID", "NAME"]
df = []
with open(scene + "/colmap_text/images.txt", "r") as fp:
    reader = csv.reader(filter(lambda row: row[0]!='#', fp), delimiter=' ')
    for i,row in enumerate(reader):
        if not i%2:
            df.append(row[:len(names)])
df = pd.DataFrame(df, columns=names)
df = df.set_index(names[:1])
cams = np.float64(df[["TX", "TY", "TZ"]].values)

In [22]:
df

,QW,QX,QY,QZ,TX,TY,TZ,CAMERA_ID,NAME
IMAGE_ID,,,,,,,,,
59,0.646869,0.00698805,-0.762142,-0.0255307,-0.372416,-0.236859,3.93023,1,r_059.png
58,0.685241,0.00826168,-0.727912,-0.0228063,-0.350841,-0.240603,3.97915,1,r_058.png
57,0.721571,0.00984317,-0.691948,-0.0211201,-0.331675,-0.240857,4.00979,1,r_057.png
56,0.756921,0.0107867,-0.653168,-0.0180317,-0.313403,-0.243235,4.03775,1,r_056.png
55,0.788593,0.00992391,-0.614627,-0.0159842,-0.305293,-0.240524,4.06148,1,r_055.png
...,...,...,...,...,...,...,...,...,...
44,0.99424,0.0218233,-0.104546,0.00897508,0.0535581,-0.268749,4.11639,1,r_044.png
45,0.987218,0.0185252,-0.158154,0.00666719,-0.000499117,-0.267154,4.1824,1,r_045.png
46,0.978942,0.0169277,-0.203395,0.00398576,-0.0327054,-0.266697,4.23957,1,r_046.png


In [7]:
num_cams = 360
# pad_cam = 4

In [8]:
r = 1.25
theta = np.linspace(0,np.pi,num_cams//2+1)
x = -r*np.cos(theta)

z = np.sqrt(r**2 - x**2)
z = np.concatenate((z,-z[1:-1][::-1]))
x = np.concatenate((x, x[1:-1][::-1]))
y = np.zeros_like(x)

In [9]:
ang_off = np.pi*0.25
ang = np.linspace(-np.pi,-2*np.pi,num_cams)[:,None] + ang_off
rw = np.cos(ang)
rxyz = np.float64([[0,1,0]]).repeat(num_cams,0)
rxyz = np.sin(ang)*rxyz
rots = np.concatenate((rw, rxyz), axis=1)

In [14]:
center = 0.5
y_off = np.abs(cams[:,1]).mean()
pos = np.dstack((x+center, y+y_off, z+center))[0]
# pos = np.concatenate((pos[-pad_cam:], pos, pos[:pad_cam]))
# rots = np.concatenate((rots[-pad_cam:], rots, rots[:pad_cam]))
y_off

0.9100126839665167

In [15]:
newp = base_cam['path'][-1].copy()
base_cam['path'] = []
for p,rt in zip(pos,rots):
    newp = newp.copy()
    newp['R'] = list(rt)
    newp['T'] = list(p)
    newp['fov'] = 80.0
    newp['scale'] = 1.
    base_cam['path'].append(newp)

In [16]:
len(base_cam['path'])

360

In [17]:
with open(scene+"/base_cam.json",'w') as f:
    json.dump(base_cam, f, indent=4)